In [1]:
!pip install -U dlt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.3/468.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 37.4 MB/s eta 0:00:00
  Created wheel for pendulum: filename=pendulum-2.1.2-cp310-cp310-manylinux_2_35_x86_64.whl size=158449 sha256=9963d0870e252aa6be

In [2]:
import dlt

pipeline=dlt.pipeline(destination='duckdb', dataset_name='mydata')
info =pipeline.run([{'id': 1, 'name': 'John'}], table_name="users")
print(info)

Pipeline dlt_colab_kernel_launcher completed in 1.92 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1699464909.339425 is LOADED and contains no failed jobs


In [3]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database 'chess_pipeline.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# list all tables
display(conn.sql("DESCRIBE"))

stats_table = conn.sql("SELECT * FROM users").df()
display(stats_table)

┌──────────────────────┬─────────┬─────────────────────┬──────────────────────┬────────────────────────────┬───────────┐
│       database       │ schema  │        name         │     column_names     │        column_types        │ temporary │
│       varchar        │ varchar │       varchar       │      varchar[]       │         varchar[]          │  boolean  │
├──────────────────────┼─────────┼─────────────────────┼──────────────────────┼────────────────────────────┼───────────┤
│ dlt_colab_kernel_l…  │ mydata  │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGIN…  │ false     │
│ dlt_colab_kernel_l…  │ mydata  │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR…  │ false     │
│ dlt_colab_kernel_l…  │ mydata  │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTA…  │ false     │
│ dlt_colab_kernel_l…  │ mydata  │ users               │ [id, name, _dlt_lo…  │ [BIGINT, VARCHAR, VARCHA…  │ false     │
└──────────────────────┴────────

,id,name,_dlt_load_id,_dlt_id
0,1,John,1699464909.339425,Mkw8ZBzxzNGKtA


In [4]:
#import dlt

# have data? dlt likes data
data = [{'id': 2, 'name': 'Ryan'}]

# open connection
pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='mydata'
)

# Upsert/merge: Update old records, insert new
load_info = pipeline.run(
    data,
    write_disposition="merge",
    primary_key="id",
    table_name="users"
)

In [5]:
# list all tables
display(conn.sql("DESCRIBE"))

stats_table = conn.sql("select * from users").df()
display(stats_table)

┌──────────────────────┬────────────────┬─────────────────────┬───┬──────────────────────┬───────────┐
│       database       │     schema     │        name         │ … │     column_types     │ temporary │
│       varchar        │    varchar     │       varchar       │   │      varchar[]       │  boolean  │
├──────────────────────┼────────────────┼─────────────────────┼───┼──────────────────────┼───────────┤
│ dlt_colab_kernel_l…  │ mydata         │ _dlt_loads          │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ dlt_colab_kernel_l…  │ mydata         │ _dlt_pipeline_state │ … │ [BIGINT, BIGINT, V…  │ false     │
│ dlt_colab_kernel_l…  │ mydata         │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ dlt_colab_kernel_l…  │ mydata         │ users               │ … │ [BIGINT, VARCHAR, …  │ false     │
│ dlt_colab_kernel_l…  │ mydata_staging │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ dlt_colab_kernel_l…  │ mydata_staging │ users               │ … │ [BIGI

,id,name,_dlt_load_id,_dlt_id
0,1,John,1699464909.339425,Mkw8ZBzxzNGKtA
1,2,Ryan,1699464917.822136,pA0jN0ANPHIfXQ
